# Vanilla Braitenberg Experiments

Author: K. Voudouris, 2023 (c) All Rights Reserved.

Contact: kv301@cam.ac.uk; k.voudouris14@googlemail.com; [Twitter @KozzyVoudouris](https://twitter.com/KozzyVoudouris); [GitHub @kozzy97](https://github.com/kozzy97)

Date: July 2023

This script runs a series of hand-coded agents on the object permanence tests and stores the results in a MySQL database. It relies on a few things:

1. All the dependencies are installed, particularly that animalai is installed properly. I recommend using a conda environment and setting up an ipykernel for running this notebook.
2. AnimalAI is installed as an executable in the `env` folder.
3. A recent installation of MySQL, configured with a database, user, and password, as well as the local (or remote) address to store on. MySQL WorkBench is a good IDE for interacting with MySQL (this was created with WorkBench 8.0)
4. A CSV file in the same directory as this notebook called `databaseConnectionDetails.csv`, containing columns `database_name`, `hostname`, `username`, and `password` for database connection, with the values in the next row. This is gitignored.

In [1]:
import numpy as np
import os
import pandas as pd
import pymysql
import random

from animalai.envs.environment import AnimalAIEnvironment
from animalai.envs.actions import AAIActions, AAIAction

import sys
sys.path.append('../src')

from vanillaBraitenbergAgent import vanillaBraitenberg #import the vanilla braitenberg class
from yamlHandling import find_yaml_files #this function finds the yaml files in a directory.
from yamlHandling import yaml_combinor #this function combines a batch of yaml files and saves the output in a temporary folder. This means we can run inference on batches of tests at once.

## Database Connection

A function for connecting to the database.

In [2]:
def databaseConnector(databaseCredentialsCSV: str):
    databaseCredentials = pd.read_csv(databaseCredentialsCSV)
    
    connection = pymysql.connect(
        host=databaseCredentials['hostname'].iloc[0],
        user=databaseCredentials['username'].iloc[0],
        password=databaseCredentials['password'].iloc[0],
        database=databaseCredentials['database_name'].iloc[0]
        )

    mycursor = connection.cursor()

    print(f"Connected to database: `{databaseCredentials['database_name'].iloc[0]}` at `{databaseCredentials['hostname'].iloc[0]}` with user `{databaseCredentials['username'].iloc[0]}`.")

    return mycursor, connection

In [3]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

mycursor.close()

print("Connection checked and closed.")

Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Connection checked and closed.


## Paths

Provide the paths to the directory containing the configs being tested over, as well as the path the animal ai environment. Finally, provide a location for generating temporary files of combined configs. This defaults to the parent directory of the github repository, to prevent results being pushed accidentally.

In [4]:
configuration_folder = "../../configs/tests_agents"

env_path = "../../env/AnimalAI"

temp_folder_location = "../../.."

## Add All Tasks In Directory To Database

Iterate through the directory and find yaml files and their task names.

In [5]:
rerunInstanceTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

yaml_files, task_names = find_yaml_files(configuration_folder)

if rerunInstanceTable:
    dropTable = "DROP TABLE IF EXISTS braitenbergvehicleinstanceresults, braitenbergvehicleintrainstanceresults, instances;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE instances(instanceid INT AUTO_INCREMENT PRIMARY KEY, instancename VARCHAR(750) UNIQUE NOT NULL);"
    mycursor.execute(sql)

for instance in task_names:
    try:
        insertQuery = "INSERT INTO instances(instancename) VALUES('" + str(instance) + "');"
        mycursor.execute(insertQuery)
        connection.commit()
    except:
        print(f"Task {instance} has already been added to this table. Moving to next.")

mycursor.close()



Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Task OP-STC-Allo-CVChick-1Occluder-Left-Beige-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-Brown-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-DarkBlue-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-DarkGreen-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-Grey-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-OffWhite-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Left-RND-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Right-Beige-0-OPQ.yml has already been added to this table. Moving to next.
Task OP-STC-Allo-CVChick-1Occluder-Ri

## Agents

Dictionaries of parameters to define some vanilla braitenberg agents. These agents have hard-coded predictable behaviours. If they detect a `GoodGoal`, `GoodGoalBounce`, `GoodGoalMulti`, or a `GoodGoalMultiBounce` in front of them, they move forwards. If they detect one of these in front of them and to the left, they execute the forwards and left action at the same time. *Mutatis mutandi* if the object is in front and to the right. If they detect a `DeathZone`, `BadGoal`, or a `BadGoalMulti` in front of them, they move backwards. If they detect one of these in front of them to the left, they move backwards and to the right. If they detect one of these in front of them and to the right, they move backwards and to the left. If they detect something in front of them, they'll continue doing that response until the end of the episode, unless it is a forwardsright or forwardsleft action, in which case they have a 0.05 probability of switching between them. If they do not detect anything in front of them, they go forwardsright or forwardsleft, with a probability of 0.05 of switching between those (and a 50/50 chance of starting with one of those actions). This encourages exploration of the space.

All of these agents are hand-coded. They do not have anything resembling object permanence. They simply go towards good things and away from bad things, exploring the space in a very simple manner if they can't see anything. These agents thus serve as excellent controls for comparison, because they can interact with the environment in a very simple way, but we can be sure that they lack the sophisticated capability of object permanence.

The agents vary in their visual acuity, which varies on 2 axes. The first is the number of rays that the agent can generate for viewing the space. The second is the number of degrees of viewing that they can see. The first agent has one ray directly in front of it, angle here is irrelevant.

In [6]:
VBB_rays_1_viewing_angle_60 = {'no_rays' : 1,
                               'angles' : 60,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Vanilla Braitenberg 1 ray'}

The next agent has 3 rays distributed over the front 60 degrees of its viewing space.

In [7]:
VBB_rays_3_viewing_angle_60 = {'no_rays' : 3,
                               'angles' : 60,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Vanilla Braitenberg 3 ray 60 degree viewing angle'}

The next agent has 3 rays distributed over the front 120 degrees of its viewing space.

In [8]:
VBB_rays_3_viewing_angle_120 = {'no_rays' : 3,
                                'angles' : 120,
                                'aai_seed' : 2023,
                                'agent_tag' : 'Vanilla Braitenberg 3 ray 120 degree viewing angle'}

5 rays over 60 degrees.

In [9]:
VBB_rays_5_viewing_angle_60 = {'no_rays' : 5,
                               'angles' : 60,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Vanilla Braitenberg 5 ray 60 degree viewing angle'}

5 rays over 120 degrees.

In [10]:
VBB_rays_5_viewing_angle_120 = {'no_rays' : 5,
                                'angles' : 120,
                                'aai_seed' : 2023,
                                'agent_tag' : 'Vanilla Braitenberg 5 ray 120 degree viewing angle'}

7 rays over 60 degrees.

In [11]:
VBB_rays_7_viewing_angle_60 = {'no_rays' : 7,
                               'angles' : 60,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Vanilla Braitenberg 7 ray 60 degree viewing angle'}

7 rays over 120 degrees.

In [12]:
VBB_rays_7_viewing_angle_120 = {'no_rays' : 7,
                                'angles' : 120,
                                'aai_seed' : 2023,
                                'agent_tag' : 'Vanilla Braitenberg 7 ray 120 degree viewing angle'}

9 rays over 60 degrees

In [13]:
VBB_rays_9_viewing_angle_60 = {'no_rays' : 9,
                               'angles' : 60,
                               'aai_seed' : 2023,
                               'agent_tag' : 'Vanilla Braitenberg 9 ray 60 degree viewing angle'}

9 rays over 120 degrees.

In [14]:
VBB_rays_9_viewing_angle_120 = {'no_rays' : 9,
                                'angles' : 120,
                                'aai_seed' : 2023,
                                'agent_tag' : 'Vanilla Braitenberg 9 ray 120 degree viewing angle'}

In [15]:
rerunAgentTable = False

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

if rerunAgentTable:
    dropTable = "DROP TABLE IF EXISTS braitenbergvehicleinstanceresults, braitenbergvehicleintrainstanceresults, braitenbergvehicles;"
    mycursor.execute(dropTable)
    
    sql = "CREATE TABLE `braitenbergvehicles` (`agentid` INT AUTO_INCREMENT PRIMARY KEY, `agent_tag` VARCHAR(300) UNIQUE, `aai_seed` INT, `no_rays` INT,  `angles` INT);"
    mycursor.execute(sql)

mycursor.close()

Connected to database: `opiaagets` at `localhost` with user `agenttester`.


In [16]:
def agentToDB (cur, dictionary : dict):
    keys = ', '.join(dictionary.keys())
    values = ', '.join(
        f"'{value}'" if not isinstance(value, (int, float, bool)) else str(int(value))
        if isinstance(value, bool) else str(value)
        for value in dictionary.values()
        )

    query = f"INSERT INTO braitenbergvehicles({keys}) VALUES ({values});"

    try:
        cur.execute(query)
    except:
        print(f"Agent `{dictionary['agent_tag']}` is already in the database.")
    
    return query

In [17]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

agentToDB(mycursor, VBB_rays_1_viewing_angle_60)

agentToDB(mycursor, VBB_rays_3_viewing_angle_60)

agentToDB(mycursor, VBB_rays_3_viewing_angle_120)

agentToDB(mycursor, VBB_rays_5_viewing_angle_60)

agentToDB(mycursor, VBB_rays_5_viewing_angle_120)

agentToDB(mycursor, VBB_rays_7_viewing_angle_60)

agentToDB(mycursor, VBB_rays_7_viewing_angle_120)

agentToDB(mycursor, VBB_rays_9_viewing_angle_60)

agentToDB(mycursor, VBB_rays_9_viewing_angle_120)

connection.commit()

mycursor.close()

Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Agent `Vanilla Braitenberg 1 ray` is already in the database.
Agent `Vanilla Braitenberg 3 ray 60 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 3 ray 120 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 5 ray 60 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 5 ray 120 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 7 ray 60 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 7 ray 120 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 9 ray 60 degree viewing angle` is already in the database.
Agent `Vanilla Braitenberg 9 ray 120 degree viewing angle` is already in the database.


## Run Inference And Store

Need to iterate through the dictionaries and run inference.

In [18]:
mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

rebuildInstanceResultsTables = True

if rebuildInstanceResultsTables:
    print("Rebuilding results tables, dropping if they already exist.")

    dropInstanceResultsTables = "DROP TABLE IF EXISTS braitenbergvehicleinstanceresults, braitenbergvehicleintrainstanceresults;"
    mycursor.execute(dropInstanceResultsTables)
    
    createInstanceTable = "CREATE TABLE braitenbergvehicleinstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, finalreward FLOAT(53), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES braitenbergvehicles(agentid), PRIMARY KEY (instanceid, agentid));"
    mycursor.execute(createInstanceTable)

    createIntraInstanceTable = "CREATE TABLE braitenbergvehicleintrainstanceresults(instanceid INT NOT NULL, agentid INT NOT NULL, step INT NOT NULL, actiontaken VARCHAR(30) NOT NULL, stepreward FLOAT(53), xvelocity FLOAT(32), yvelocity FLOAT(32), zvelocity FLOAT(32), xpos FLOAT(32), ypos FLOAT(32), zpos FLOAT(32), FOREIGN KEY (instanceid) REFERENCES instances(instanceid), FOREIGN KEY(agentid) REFERENCES braitenbergvehicles(agentid), PRIMARY KEY(instanceid, agentid, step));"
    mycursor.execute(createIntraInstanceTable)

    print("Tables `braitenbergvehicleinstanceresults` and `braitenbergvehicleintrainstanceresults` have been successfully built.")

mycursor.close()


Connected to database: `opiaagets` at `localhost` with user `agenttester`.
Rebuilding results tables, dropping if they already exist.
Tables `braitenbergvehicleinstanceresults` and `braitenbergvehicleintrainstanceresults` have been successfully built.


Define a function to run the experiments. This takes an agent dictionary and first checks whether any results have been recorded for it. If not, then it proceeds with testing. It does testing in batches, generating a temporary yml file to run training on and storing the final episode reward, as well as the intra-instance results.

In [19]:
def runBraitenbergAndStore(cur, con, batch_size: int, agent_dict: dict, yaml_files, task_names, temp_folder_location, agent_inference = False):
    
    # first, check if this agent has been added to the DB already

    select_agent_id = f"SELECT DISTINCT agentid FROM braitenbergvehicles WHERE agent_tag = '{agent_dict['agent_tag']}';"

    cur.execute(select_agent_id)

    agentid = int(cur.fetchone()[0]) #fetch the result, there should just be 1.

    # now proceed with testing
    port = 5500 + random.randint( #create random base port.
        0, 2000
        )
        
    batch_counter = 0

    #set seed
    random.seed(agent_dict['aai_seed'])

    for yaml_index in range(0, len(yaml_files), batch_size):

        if ((yaml_index + batch_size)-1) > len(yaml_files) or batch_size > len(yaml_files):
            upper_bound = len(yaml_files)
        else:
            upper_bound = ((yaml_index + batch_size)-1)

        print(f"Running inferences on batch {batch_counter + 1} of {upper_bound+1} files of total {len(yaml_files)}.")

        batch_files = yaml_files[yaml_index:upper_bound]

        batch_file_names = task_names[yaml_index:upper_bound]

        batch_temp_file_name = f"TempConfig_{yaml_index}.yml"

        config_file_path = yaml_combinor(file_list = batch_files, temp_file_location=temp_folder_location, stored_file_name = batch_temp_file_name)

        agent = vanillaBraitenberg(agent_dict['no_rays']) # initialise agent class

        print("Opening AAI Environment.")

        temp_port = port + yaml_index # increment through ports to prevent calling the same socket.

        aai_env = AnimalAIEnvironment( 
            inference=agent_inference, #Set true when watching the agent
            seed = agent_dict['aai_seed'],
            worker_id=agent_dict['aai_seed'],
            file_name=env_path,
            arenas_configurations=config_file_path,
            base_port=temp_port,
            useCamera=False,
            useRayCasts = True,
            raysPerSide=int((agent_dict['no_rays'])/2),
            rayMaxDegrees=agent_dict['angles']
        )

        behavior = list(aai_env.behavior_specs.keys())[0] # by default should be AnimalAI?team=0

        firststep = True

        for _instance in range(len(batch_files)): 

            #get instance ID
            getInstanceIDQuery = f"SELECT instanceid FROM instances WHERE instancename = '{batch_file_names[_instance]}';" #get the instance ID for 

            cur.execute(getInstanceIDQuery)
       
            instanceid = int(cur.fetchone()[0]) #fetch the result, there should just be 1.

            check_agent_instance_query = f"SELECT CASE WHEN EXISTS (SELECT * FROM braitenbergvehicleinstanceresults WHERE agentid = {agentid} AND instanceid = {instanceid}) THEN 1 ELSE 0 END AS AnyData;"

            cur.execute(check_agent_instance_query)

            check_agent_instance = int(cur.fetchone()[0])

            if check_agent_instance == 0:

                #prepare to run instance

                if firststep:
                    aai_env.step() # take first step to get an observation
                    firststep = False
                    
                dec, term = aai_env.get_steps(behavior)

                done = False

                episodeReward = 0

                step_counter = 0
    
                while not done:
                        
                    observations = aai_env.get_obs_dict(dec.obs)

                    raycasts = observations["rays"] # Get the raycast data

                    action = agent.get_action(raycasts)

                    aai_env.set_actions(behavior, action.action_tuple)

                    aai_env.step()

                    step_counter += 1

                    dec, term = aai_env.get_steps(behavior)

                    if len(dec.reward) > 0 and len(term) <= 0:
                        episodeReward += dec.reward
                        try:
                            intraInstanceQuery = f"INSERT INTO braitenbergvehicleintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, '{action.name}', {float(episodeReward)}, {observations['velocity'][0]}, {observations['velocity'][1]}, {observations['velocity'][2]}, {observations['position'][0]}, {observations['position'][1]}, {observations['position'][2]});"
                            cur.execute(intraInstanceQuery)
                            con.commit()
                            
                        except:
                            print(intraInstanceQuery)

                    elif len(term) > 0: #Episode is over
                        episodeReward += term.reward
                        print(f"Episode Reward: {episodeReward}")
                        done = True
                        firststep = True

                        try:
                            intraInstanceQuery = f"INSERT INTO braitenbergvehicleintrainstanceresults(instanceid, agentid, step, actiontaken, stepreward, xvelocity, yvelocity, zvelocity, xpos, ypos, zpos) VALUES ({instanceid}, {agentid}, {step_counter}, '{action.name}', {float(episodeReward)}, {observations['velocity'][0]}, {observations['velocity'][1]}, {observations['velocity'][2]}, {observations['position'][0]}, {observations['position'][1]}, {observations['position'][2]});"
                            cur.execute(intraInstanceQuery)
                            con.commit()
                            
                        except:
                            print(intraInstanceQuery)
                             
                        try:
                            insertInstanceResults = f"INSERT INTO braitenbergvehicleinstanceresults(instanceid, agentid, finalreward) VALUES ({instanceid}, {agentid}, {float(episodeReward)});"
                            cur.execute(insertInstanceResults)
                            con.commit()
                            print("Pushing results to database.")
                        except:
                            print("It looks like this agent has already been tested on this instance.")
                    
                    else:
                        pass

            else:
                print("This instance has already been tested on. Skipping so we don't record intrainstance results accidentally (e.g., if they hit lava early on a previous run.).")
                done = False

                if firststep:
                    aai_env.step() # take first step to get an observation
                    firststep = False
                    
                dec, term = aai_env.get_steps(behavior)

                while not done:
                    aai_env.set_actions(behavior, agent.actions.NOOP.action_tuple)
                    aai_env.step()
                    dec, term = aai_env.get_steps(behavior)
                    if len(term) > 0: #Episode is over
                        done = True
                        firststep = True
                
                print("Moving to next instance")
                


                
        aai_env.close()
        print("Moving to next batch.")
        batch_counter += 1
    

In [ ]:
agent_dict_list = [VBB_rays_1_viewing_angle_60, VBB_rays_3_viewing_angle_60, VBB_rays_3_viewing_angle_120, VBB_rays_5_viewing_angle_60, VBB_rays_5_viewing_angle_120, VBB_rays_7_viewing_angle_60, VBB_rays_7_viewing_angle_120, VBB_rays_9_viewing_angle_60, VBB_rays_9_viewing_angle_120]

yaml_batch_size = 100

mycursor, connection = databaseConnector('databaseConnectionDetails.csv')

for agent_dictionary in agent_dict_list:

    print(f"Running {agent_dictionary['agent_tag']}")

    runBraitenbergAndStore(mycursor, connection, yaml_batch_size, agent_dict=agent_dictionary, yaml_files=yaml_files, task_names=task_names, temp_folder_location=temp_folder_location, agent_inference=False)


mycursor.close()